In [9]:
#Right now we could maybe widdle down to movies above a certain rating + number of ratings

In [2]:
import pandas as pd
import csv
import time
import numpy as np
import itertools
df_ratings = pd.read_csv('data/ratings.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep='\t')

,tconst,averageRating,numVotes
0,tt0000001,5.6,1696
1,tt0000002,6.0,210
2,tt0000003,6.5,1448
3,tt0000004,6.1,122
4,tt0000005,6.1,2245


In [6]:
movies = titles[titles['titleType']=='movie']
movies = list(movies['tconst'].unique())
print(len(movies))

573966


In [7]:

#read in the ratings data
print(np.mean(df_ratings['numVotes']))
print(np.mean(df_ratings['averageRating']))
avgVotes = np.mean(df_ratings['numVotes'])
df_ratings.head()
qualifiedMovies = []
len(df_ratings)
boolean_series = df_ratings.tconst.isin(movies)
df_ratings = df_ratings[boolean_series]
for row in df_ratings.iterrows():
    if row[1]['averageRating'] >= 8.5 and row[1]['numVotes'] >= avgVotes:
        qualifiedMovies.append(row[1]['tconst'])
    

955.1516708090234
6.900688579030731


In [27]:
qualifiedMoviesFinal = []
for movie in qualifiedMovies:
    year = (list(titles[titles['tconst']==movie]['startYear'])[0])
    if type(year) == str:
        year = int(year)
    if year>= 2000:
        qualifiedMoviesFinal.append(movie)
print(qualifiedMoviesFinal)

['tt0120737', 'tt0167260', 'tt0167261', 'tt0172495', 'tt0245429', 'tt0253474', 'tt0270321', 'tt0271383', 'tt0311594', 'tt0314067', 'tt0317248', 'tt0319836', 'tt0339535', 'tt0346336', 'tt0358456', 'tt0367495', 'tt0375093', 'tt0383846', 'tt0400234', 'tt0407887', 'tt0416960', 'tt0419781', 'tt0459516', 'tt0468569', 'tt0480732', 'tt0482571', 'tt0770802', 'tt0816692', 'tt0829269', 'tt0892425', 'tt0965382', 'tt1014762', 'tt10189514', 'tt10370116', 'tt10469118', 'tt10687070', 'tt10702760', 'tt10765852', 'tt1094594', 'tt11191124', 'tt1152758', 'tt11581174', 'tt1165293', 'tt11989890', 'tt12361178', 'tt12865416', 'tt13177004', 'tt13243898', 'tt13458600', 'tt1375666', 'tt1424432', 'tt1496729', 'tt1601792', 'tt1621830', 'tt1675434', 'tt1754109', 'tt1795369', 'tt2077886', 'tt2083271', 'tt2170667', 'tt2185022', 'tt2412748', 'tt2414166', 'tt2415372', 'tt2473476', 'tt2582802', 'tt2592910', 'tt3313066', 'tt3327994', 'tt3557258', 'tt3590482', 'tt3674140', 'tt3917908', 'tt3970482', 'tt4476736', 'tt4519488

In [28]:
print(len(qualifiedMovies),len(qualifiedMoviesFinal))

272 126


In [50]:

#[lambda x: x['tconst'] in qualifiedMovies]

In [30]:
count = 0
freqActors = dict()
for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
    chunk = chunk[chunk['category']=='actor']
    chunk = chunk.groupby('tconst')
    for name, group in chunk:
        if name not in qualifiedMovies:
            continue
        else:
            actors = group['nconst'].unique()
            for actor in actors:
                if actor in freqActors:
                    freqActors[actor][0] += 1
                    freqActors[actor][1].append(name)
                else:
                    freqActors[actor] = [1,[name]]
    

In [49]:
freqActorsFinal = dict()
for actor in freqActors:
    if freqActors[actor][0] >= 2:
        freqActorsFinal[actor] = freqActors[actor]

In [55]:
freq_pairs = dict()
for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
    chunk = chunk[chunk['category']=='actor']
    chunk = chunk.groupby('tconst')
    for name, group in chunk:
        freqActorsGroup = [actor for actor in group['nconst'] if actor in freqActorsFinal]
        freqPairsGroup = list(itertools.combinations(freqActorsGroup,2))
        for pair in freqPairsGroup:
            if pair in freq_pairs:
                freq_pairs[pair][0] += 1
                freq_pairs[pair][1].append(name)
            else:
                freq_pairs[pair] = [1,[name]]

In [58]:
print(freq_pairs)

{('nm0017988', 'nm0904303'): [1, ['tt0053785']], ('nm0660233', 'nm0705880'): [3, ['tt0057544', 'tt0121253', 'tt0123391']], ('nm0135908', 'nm0660233'): [3, ['tt0061168', 'tt0123390', 'tt0123391']], ('nm0154170', 'nm0315917'): [3, ['tt0063023', 'tt0080856', 'tt0094069']], ('nm0135908', 'nm0705880'): [5, ['tt0063168', 'tt0123390', 'tt0123391', 'tt0123392', 'tt0123829']], ('nm0631877', 'nm0073353'): [5, ['tt0071028', 'tt0074039', 'tt0075094', 'tt0076276', 'tt1199764']], ('nm0000199', 'nm0000134'): [3, ['tt0071562', 'tt0113277', 'tt0809488']], ('nm0904303', 'nm0631877'): [17, ['tt0074039', 'tt0075094', 'tt0076276', 'tt0920154', 'tt0920155', 'tt0920156', 'tt0931047', 'tt0936314', 'tt0936315', 'tt0943088', 'tt0943089', 'tt0943090', 'tt0943091', 'tt0943092', 'tt0943093', 'tt0943094', 'tt1360141']], ('nm0904303', 'nm0073353'): [4, ['tt0074039', 'tt0075094', 'tt0076276', 'tt1360141']], ('nm0904303', 'nm0831262'): [1, ['tt0076276']], ('nm0631877', 'nm0831262'): [15, ['tt0076276', 'tt0084115', 'tt

In [78]:
def getSoloMovies(name1,name2,pairsDict,soloDict):
    name1solomovies = []
    name2solomovies = []
    for movie in soloDict[name1][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name1solomovies.append(movie)
    for movie in soloDict[name1][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name2solomovies.append(movie)
    return name1solomovies,name2solomovies
def getRatings(movies):
    ratings = []
    for movie in movies:
        ratings.append(list(df_ratings[df_ratings['tconst']== movie]['averageRating'])[0])
    rating = np.mean(ratings)
    return rating

In [79]:
freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
#('nm0482320', 'nm0007123'): 32,
count = 0
freq_pairsFinal = dict()
for pair in freq_pairs:
    if (pair[1],pair[0]) not in freq_pairsFinal:
        freq_pairsFinal[pair] = freq_pairs[pair]
print(len(freq_pairsFinal))
for pair in freq_pairsFinal:
    print(pair)
    break
print(freqActorsFinal['nm0482320'])
movies1,movies2 = getSoloMovies('nm0482320','nm0893449',freq_pairsFinal,freqActorsFinal)

print(getRatings(movies1))

177
('nm0482320', 'nm0893449')
[11, ['tt0119385', 'tt0133826', 'tt0214585', 'tt0214915', 'tt0237376', 'tt0249188', 'tt0255306', 'tt0282778', 'tt0291855', 'tt0292246', 'tt12361178']]
8.644444444444444


In [15]:
names = pd.read_csv('data/names.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep = '\t')

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
names[names['nconst']=='nm0007123']['primaryName']
#names[names['nconst']=='nm0007123']
#titles[titles['tconst']== 'tt0118655']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
7096,nm0007123,Mammootty,1951,\N,"actor,producer,music_department","tt1573478,tt0097851,tt0230597,tt0271648"


In [33]:
titles['titleType'].unique()


array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'episode'],
      dtype=object)